In [2]:
pip install rasterio numpy


  Using cached affine-2.4.0-py3-none-any.whl.metadata (4.0 kB)
  Using cached cligj-0.7.2-py3-none-any.whl.metadata (5.0 kB)
  Using cached snuggs-1.4.7-py3-none-any.whl.metadata (3.4 kB)
  Using cached click_plugins-1.1.1-py2.py3-none-any.whl.metadata (6.4 kB)
   ---------------------------------------- 0.0/24.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/24.3 MB 682.7 kB/s eta 0:00:36
   ---------------------------------------- 0.0/24.3 MB 660.6 kB/s eta 0:00:37
   ---------------------------------------- 0.1/24.3 MB 563.7 kB/s eta 0:00:44
   ---------------------------------------- 0.1/24.3 MB 819.2 kB/s eta 0:00:30
   ---------------------------------------- 0.2/24.3 MB 952.6 kB/s eta 0:00:26
   ---------------------------------------- 0.3/24.3 MB 1.2 MB/s eta 0:00:20
    --------------------------------------- 0.3/24.3 MB 1.3 MB/s eta 0:00:19
    --------------------------------------- 0.4/24.3 MB 1.4 MB/s eta 0:00:18
    --------------------------------------

In [2]:
import os
import numpy as np
import rasterio
from rasterio.enums import Resampling

# Define paths
input_directory = r'D:\for interns\15% overlap detect\1'
output_directory = r'D:\for interns\no black 15%\1'

# Ensure the output directory exists
os.makedirs(output_directory, exist_ok=True)

# Define a threshold for black pixels
black_threshold = 0.90  # 90% of the pixels

def is_mostly_black(image, threshold=black_threshold):
    # Calculate the number of black pixels (assuming black is pixel value 0 for all bands)
    black_pixels = np.sum(np.all(image == 0, axis=0))
    total_pixels = image.shape[1] * image.shape[2]
    # Calculate the proportion of black pixels
    black_pixels_ratio = black_pixels / total_pixels
    return black_pixels_ratio > threshold

# Iterate over each chunk
for chunk_filename in os.listdir(input_directory):
    chunk_path = os.path.join(input_directory, chunk_filename)
    
    # Check if the file is indeed a file and not a directory
    if not os.path.isfile(chunk_path):
        continue
    
    try:
        with rasterio.open(chunk_path) as src:
            chunk_data = src.read()  # Read the entire chunk

            if not is_mostly_black(chunk_data):
                output_path = os.path.join(output_directory, chunk_filename)
                with rasterio.open(output_path, 'w', **src.profile) as dst:
                    dst.write(chunk_data)
    except rasterio.errors.RasterioIOError as e:
        print(f"Error opening {chunk_path}: {e}")
    except PermissionError as e:
        print(f"Permission denied for {chunk_path}: {e}")

print("Completed filtering black images.")


Completed filtering black images.
